In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.float_format = '{:.2f}'.format
pd.options.mode.chained_assignment = None

# Анализ товарооборота России только со странами АТР

In [3]:
TCBT = pd.read_csv(r"/Users/kosarenok/Downloads/DATTSVT 2.csv", sep= "\t",on_bad_lines="skip")
TCBT

/var/folders/n0/z4q56_291yl18hx7bzl2b5b40000gn/T/ipykernel_19077/2087126616.py:1: DtypeWarning: Columns (3,7) have mixed types. Specify dtype option on import or set low_memory=False.
  TCBT = pd.read_csv(r"/Users/kosarenok/Downloads/DATTSVT 2.csv", sep= "\t",on_bad_lines="skip")


,napr,period,nastranapr,tnved,edizm,Stoim,Netto,Kol,Region,Region_s
0,ЭК,02/2019,US,85,NaN,"14850,8",99,0,75000 - ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,05-УРАЛЬСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ
1,ИМ,01/2019,CN,57,55.00,"10014,27",4137,"2584,88",04000 - КРАСНОЯРСКИЙ КРАЙ,06-СИБИРСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ
2,ИМ,01/2019,CN,57,55.00,"194512,94","88926,55","58870,71",05000 - ПРИМОРСКИЙ КРАЙ,07-ДАЛЬНЕВОСТОЧНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ
3,ИМ,01/2019,CN,57,55.00,"145648,27","68007,03",41792,40000 - ГОРОД САНКТ-ПЕТЕРБУРГ ГОРОД ФЕДЕРАЛЬНО...,02-СЕВЕРО-ЗАПАДНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ
4,ИМ,01/2019,CN,57,55.00,"30148,45","4265,7","2511,96",41000 - ЛЕНИНГРАДСКАЯ ОБЛАСТЬ,02-СЕВЕРО-ЗАПАДНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ
...,...,...,...,...,...,...,...,...,...,...
2457034,ЭК,01/2022,SE,15,NaN,25344,21300,0,46000 - МОСКОВСКАЯ ОБЛАСТЬ,01-ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ
2457035,ИМ,01/2022,SE,15,NaN,18624,"4806,144",0,38000 - КУРСКАЯ ОБЛАСТЬ,01-ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ
2457036,ЭК,01/2022,RS,15,NaN,"132647,08","99399,488",0,46000 - МОСКОВСКАЯ ОБЛАСТЬ,01-ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ
2457037,ЭК,01/2022,RS,15,NaN,"206551,21",154450,0,14000 - БЕЛГОРОДСКАЯ ОБЛАСТЬ,01-ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ


In [4]:
list_of_countries = ["AU", "BN", "VN", "HK", "ID", "CA", "CN", "KR",
                      "MY", "MX", "NZ", "PG" "PE", "SG", "US", "TH",
                        "TW", "PH", "CL", "JP"]

list_of_needed_countries = ["AU", "VN", "ID", "CA", "CN",
                             "KR", "MX", "SG", "US","JP"]

In [5]:
TCBT['Stoim'] = [float(str(i).replace(",",".")) for i in TCBT['Stoim']]

TCBT["period"] = pd.to_datetime(TCBT["period"])

TCBT["Year"] = TCBT["period"].dt.year

In [6]:
tbct_apr_pivot = (TCBT
                  .query("nastranapr in @list_of_countries")
                  .pivot_table(values = 'Stoim', index = ['nastranapr'], 
                               columns = ['Year','napr'], aggfunc = np.sum)
                  .to_excel("diploma_analysis_wtf.xlsx", sheet_name="Россия и АТР")
                  )


# Анализ товарооборота России со всеми странами

In [7]:
tbct_world_pivot = (TCBT
                    .pivot_table(values = 'Stoim', index = ['nastranapr'], 
                                 columns = ['Year','napr'], aggfunc = np.sum)
                    .to_excel("diploma_analysis.xlsx", sheet_name="Россия и Мир")
                    )



# Удельный вес ведущих стран АТР в обороте, импорте и экспорте внешней товарной торговли РФ

In [8]:
tcbt_apr_2021 = (TCBT
                 .query("Year == 2021")
                 .rename(columns={"nastranapr" : "city"})
                 .groupby("city")["Stoim"].sum()
                 .to_frame()
                 )

In [9]:
tcbt_apr_2021 = (tcbt_apr_2021
                 .assign(ves = (tcbt_apr_2021.Stoim / tcbt_apr_2021.Stoim.sum()) * 100)
                 .query("city in @list_of_countries")
                 .to_excel("aboba.xlsx", sheet_name="test")
                )

In [10]:
tcbt_apr_2021_ei = (TCBT
                    .query("Year == 2021")
                    .rename(columns={"nastranapr" : "city"})
                    .pivot_table(values = "Stoim", index = ["city"],
                                  columns = ["napr"], aggfunc = np.sum)
                    )

tcbt_apr_2021_ei = (tcbt_apr_2021_ei
                    .assign(import_ves = (tcbt_apr_2021_ei.ИМ / tcbt_apr_2021_ei.ИМ.sum()) * 100,
                            export_ves = (tcbt_apr_2021_ei.ЭК / tcbt_apr_2021_ei.ЭК.sum()) * 100)
                    .query("city in @list_of_needed_countries")
                    .to_excel("aboba.xlsx", sheet_name="test")
                    )

# Товарная структура внешней торговли РФ

In [26]:
tcbt_tnved_2021_apr = (TCBT
                   .query("nastranapr in @list_of_countries")
                   .pivot_table(values = 'Stoim', index = ['tnved'], 
                                columns = ['napr'], aggfunc = np.sum)
                   #.to_excel("aboba.xlsx", sheet_name="test")
                   )

In [24]:
tcbt_tnved_2021 = (TCBT
                   .pivot_table(values = 'Stoim', index = ['tnved'], 
                                columns = ['napr'], aggfunc = np.sum)
                   .to_excel("aboba.xlsx", sheet_name="test")
                   )